# **현재 진행상황** 
### **1. 데이터 불러오기** 
    1) 크롤링한 데이터셋을 pandas library를 통해 pandas.DataFrame 형태로 읽는다. 
    2) 'TAGS' 컬럼만 따로 pandas.Series의 변수로 저장한다.  

        'TAGS'열의 문제점:
        - 태그들의 집합을 리스트의 형태로 저장하려 한듯하나, 리스트로 인식하지 못한다.
        - 각 행의 모든 값이 char 타입으로 인식된다. 
 

### **2. 연관분석에 맞는 데이터 형태(transaction)로 변환 - 1단계 **
    1) 'TAGS'열 문제 해결을 위한 함수: def dataset(pandas_series)
        - 매개변수로 pandas_series 변수를 넣고, 리턴 값으로 [[태그집합 1], [태그집합2], ... ,[태그집합n]]를 반환한다.
    
### **3. 광고성 글 제거 **
    1) 광고성 글을 파악하기 위한 함수: def del_ad(df)
        - 하나의 태그집합 안에 300개가 넘는 태그가 속한 경우가 있었고, 이는 음식점 및 성인 광고성 글로 판단되었다. 
        - 하나의 태그집합 안에 30개가 넘는 태그가 속한 리스트의 인덱스를 반환하는 함수이다. 
    2) 광고성 글을 제거하는 함수: def dataset_del_ad(df)
        - 위의 함수를 통해 파악된 인덱스의 데이터를 제거하는 함수이다. 
    
### **4. 연관분석에 맞는 데이터 형태(transaction)로 가공 - 2단계 **
    1) transaction 데이터 형태의 각 행에는 def dataset(pandas_series)의 결과로 나온 각 태그집합이 들어간다. 
    2) 이때 각 열에는 하나의 태그씩만 들어가야 한다. 열의 길이는 크롤링한 데이터의 수(약 30만 개)이며, 행의 길이는 30개이다. 
    3) 또한 sparse한 공간은 모두 None값으로 채워줘야 한다. 
 
     <최종 transaction 데이터 형태의 모습 >
    
                1행    2행     3행     4행    5행    ... 30행
        1열     태그1,  태그2,  태그3,  태그4,  None, ..., None
        2열     태그1,  태그2,  None,  None,  None, ..., None
        3열     태그1,  None,  None,  None,  None, ..., None
        4열     태그1,  태그2,  태그3,  태그4,  태그5, ..., 태그30
        .
        .
        .
        30만열   태그1, .............................., None 
    
# **앞으로의 방향성**
### **5. 연관분석을 위한 실험 진행** 
    1) support, confidence, lift, minlen 값을 바꿔가보며 실험해야 한다. 

In [1]:
import pandas as pd
from pandas import DataFrame as df
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from apyori import apriori

In [2]:
insta1 = pd.read_excel('instagram_soldrink_1.xlsx')
# final = pd.read_csv('end_instagram_soldrink.csv') 

In [3]:
# 태그 따로 분리
tag = [insta1['TAGS']][0]      

In [4]:
def dataset(pandas_series):
    res = []
    for i in range(len(pandas_series)):
        res = res + [pandas_series[i].replace("[","").replace("]","").replace("'","").replace('\n',"").replace(",","").replace("#","").split(" ")]  
    return res  # type: list

def del_ad(df):
    index_delete = []
    for i in range(len(df)):
        if len(df[i]) > 30:
            index_delete = index_delete + [df.index(df[i])]
    index_delete = list(set(index_delete))
    index_delete.sort()
    return index_delete

def dataset_del_ad(df):
    for i in range(len(df)):
        if i in del_ad(df):
            del df[i]
            i = i - 1
        else:
            pass 

# 2차원 배열 df의 각 리스트의 길이 중 가장 긴 길이를 반환하는 함수
# 이걸 돌려보니 한 태그 집합 안에 300개가 넘는 태그가 있다는 것을 알 수 있었다. 
# def t_column(df):
#     col=[]
#     for i in range(len(df)): 
#         b = len(df[i]) 
#         col = col + [b]

#     return max(col) 

def transact(df):
    df = pd.DataFrame(df)
    transaction = []
    for i in range(0, len(df)):
        transaction.append([str(df.values[i,j]) for j in range(0,50)])

    return transaction


In [5]:
data = dataset(tag) 
wierdo = del_ad(data)
for i in wierdo:
    print(data[i])

['selca', 'selfie', 'dailyphoto', 'dailypic', 'insta', 'boy', '인친', '소통', '좋아요', '선팔', '먹스타그램', '먹스타', '치킨', '치맥', '혼술', 'like4like', 'l4l', 'f4f', 'fff', 'lfl', '다이렉트', 'ootd', '일상', '소통', 'like', 'daily', '일상', '성남', '강남', '모란', '맞팔해용', '맞팔🙌']
['대전호빠', '대전호스트빠', '둔산동', '유성', '호빠', '호스트바', '선수', '가라오케', '충남', '생파', '선수', '호빠선수', '술자리', '청주', '대구', '불토', '불금', '술생각', '남자', '서울', '세종', '부산', '충북', '계룡', '강남', '2차', '맨빠', '혼술', '천안', '노래방', '회식', '가즈아']
['리릭쿠마', '커피', '사탕', '화이트데이', '봄', '네일아트', '새단장', '@biv7777', '네일뷰티인뷰', '@allabouthn', '먹방스타그램', '먹스타그램', '선릉네일', '요가', '널스타그램', '맛스타그램', '존예', '강남맛집', '존맛', '존맛탱', '혼술', '치맥', '승무원', '강남술집', '맛집스타그램', '뷰티인뷰', '셀카', '맛집그램', '선릉역네일샵', '대치동네일', '경리단길', '경리단네일']
['대전호빠', '대전호스트빠', '둔산동', '유성', '호빠', '호스트바', '선수', '가라오케', '충남', '생파', '선수', '호빠선수', '술자리', '청주', '대구', '불토', '불금', '술생각', '남자', '서울', '세종', '부산', '충북', '계룡', '강남', '맨빠', '혼술', '천안', '노래방', '회식', '가즈아']
['selenator', 'selenamariegomez', 'selena', 'selenagomez', 'selenagomezedit', 's

In [7]:
data = dataset(tag)  # 모든 태그들을 인덱싱이 가능한 리스트로 반환
dataset_del_ad(data) # 태그가 30개 이상인 행 제거하기 

In [8]:
transactions = transact(data) #마지막 데이터셋 

In [10]:
# 제대로 인덱싱 되는지 확인 
transactions[0][0] 
transactions[0][1]
len(transactions)
#transactions = putnone(transactions)

'발렌타인17년산'

'귤'

9713

In [14]:
file = pd.DataFrame(transactions)
file.to_csv('file.csv', sep=',', index=False)

# 실험 시작 (여기서부터 안돌아감) 
min_support: 1 / len(data)*50 = 0.005
min_confidence = 0.2
min_lift = 1
min_length = 2

In [ ]:
rules = apriori(transactions, min_support = 0.01, min_confidence = 0.1, min_lift = 1.5, min_length = 2)
results = list(rules)

In [ ]:
rules2 = apriori(transactions, min_support = 0.001, min_confidence = 0.1, min_lift = 1, min_length = 2)
results2 = list(rules2)